In [1]:
import pandas as pd
import sqlite3
import numpy as np

In [8]:
# musical features
style = pd.read_csv('../deprecated/style.csv', header=None)
style.columns = ['track_id','genre']
style.loc[:, 'has_musical_ftr'] = 1
style.head()

,track_id,genre,has_musical_ftr
0,TRAAAAK128F9318786,Metal_Alternative,1
1,TRAAAAV128F421A322,Punk,1
2,TRAAAAW128F429D538,Hip_Hop_Rap,1
3,TRAAACV128F423E09E,Rock_Neo_Psychedelia,1
4,TRAAAEF128F4273421,Pop_Indie,1


In [10]:
# covers
cover = pd.read_csv('../deprecated/cover_tid_lookup.csv')
cover.loc[:,'has_cover_img'] = 1
del cover['tags']
cover.head()

,track_id,image_name,has_cover_img
0,TRMXFDC128F428D430,1e0749cb-19f3-40e8-9852-7d4bafc48bfc,1
1,TRLDFYW128F428D425,1e0749cb-19f3-40e8-9852-7d4bafc48bfc,1
2,TRMMMUT128F42646E8,add26870-a2b1-4d18-ae39-804551ef75a4,1
3,TRWWVQV128F4264E27,ca4634f5-6694-4bae-aac7-35df2c1c82db,1
4,TRWVYYD128F4273E27,ca4634f5-6694-4bae-aac7-35df2c1c82db,1


In [5]:
cover_rank = (cover.loc[cover['image_name']!=0, ['image_name','track_id']]
                   .drop_duplicates()
                   .reset_index(drop=True)
                   .reset_index())
cover_rank = cover_rank.rename(columns={'index':'cover_id'})
cover_rank.loc[:, 'cover_id'] = cover_rank.loc[:, 'cover_id'].astype(int)
cover_rank.loc[:, 'cover_id'] = cover_rank.groupby('image_name')['cover_id'].transform(lambda x: x.min())

In [8]:
lyr = pd.read_csv('/storage/thesis/lyrics/lyrics_piv_trunc.csv', usecols=['track_id']).assign(has_lyrics = 1)
lyr.head()

,track_id,has_lyrics
0,TRAAAAV128F421A322,1
1,TRAAAEF128F4273421,1
2,TRAAAHJ128F931194C,1
3,TRAAAHZ128E0799171,1
4,TRAAAJG128F9308A25,1


In [9]:
dt = style.merge(lyr, on=['track_id'], how='left')
dt = dt.merge(cover, on=['track_id'], how='left')
dt = dt.merge(cover_rank.loc[:,['track_id','cover_id']], on=['track_id'], how='left')
idx_cols = ['has_musical_ftr','has_lyrics','has_cover_img']
dt.loc[:, idx_cols] = dt.loc[:, idx_cols].fillna(0)
# dt.loc[:, 'track_cnt'] = dt.groupby(['cover_id'])['track_id'].transform(lambda x: x.nunique())

In [10]:
dt.groupby(['has_musical_ftr','has_lyrics','has_cover_img'])['track_id'].nunique() / dt.shape[0]

has_musical_ftr  has_lyrics  has_cover_img
1                0.0         0.0              0.74210
                             1.0              0.00019
                 1.0         1.0              0.25771
Name: track_id, dtype: float64

In [11]:
dt.groupby(['has_musical_ftr','has_lyrics','has_cover_img'])['track_id'].nunique()

has_musical_ftr  has_lyrics  has_cover_img
1                0.0         0.0              203288
                             1.0                  52
                 1.0         1.0               70596
Name: track_id, dtype: int64

In [12]:
# 70,000 samples have both lyrics, musical features, cover images, AND a genre label

In [31]:
data = dt.copy()
data = data.loc[(data['has_lyrics']==1)&(data['has_cover_img']==1), :]

np.random.seed(1234)
covers = data['cover_id'].unique()
np.random.shuffle(covers)
train, test = covers[:int(len(covers)*.7)], covers[int(len(covers)*.7)+1: int(len(covers)*.85)]

data.loc[:, 'is_test'] = data['cover_id'].map(lambda s: 1 if s in test else (0 if s in train else 2))

In [34]:
data.to_csv('test_train_split.csv', index=False)

In [35]:
pd.read_csv('test_train_split.csv').head()

,track_id,genre,has_musical_ftr,has_lyrics,image_name,has_cover_img,cover_id,is_test
0,TRAAAAV128F421A322,Punk,1,1.0,4b3cc7ae-654e-4408-89e1-4ea70cf86717,1.0,70206.0,0
1,TRAAAEF128F4273421,Pop_Indie,1,1.0,c1f9b340-a37e-4dc8-aaf9-f622c5457f1a,1.0,21597.0,0
2,TRAAAHJ128F931194C,Pop_Indie,1,1.0,dc9b75f9-d5e7-434a-82c4-0d4e9790efd4,1.0,1330.0,0
3,TRAAAHZ128E0799171,Hip_Hop_Rap,1,1.0,62a1cd53-1339-4f04-8113-287b174c3917,1.0,33631.0,2
4,TRAAAJG128F9308A25,Country_Traditional,1,1.0,0cee21c0-9657-469c-9a72-b81bacc88eee,1.0,40306.0,1
